In [1]:
# Import all functions necessary for the notebook
import csv
from math import sqrt
from random import randrange
from random import seed

In [2]:
##### Loading, and conversion of CSV file #####

## Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

## Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

## Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup


##### Normalize Data ###########

# Find the min and max values for each column

def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        colvalues = [row[i] for row in dataset]
        min_value = min(colvalues) 
        max_value = max(colvalues)
        minmax.append([min_value, max_value])
    return minmax

# Normalize the dataset except last row for classification values
def Normalize_Dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [3]:
#### Split dataset into k folds and algorithm procedure

# Split a dataset into $k$ folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Evaluate an algorithm using a cross-validation split
def evaluate_algorithm_cv(dataset, algorithm, n_folds, performance_assessment, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        performance = performance_assessment(actual, predicted)
        scores.append(performance)
    return scores

In [4]:
#### Performance measure #####

# Get accuracy of prediction #
def getAccuracy(actual,predicted):
    correct = 0
    for x in range(len(actual)):
        if actual[x] == predicted[x]:
            correct += 1
    return correct / float(len(actual)) * 100.00

In [5]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

# GINI index as cost function to minimize
def gini_index(groups,classes):
    #count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    #Gini indexes weighted sum
    Gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0.0:
            continue
        score = 0.0
        #score group based on score of each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p*p
        #weight group score b yrelative size
        Gini += (1.0-score) * (size / n_instances)
    return Gini

#Split a dataset
def test_split(index,value,dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index]< value:
            left.append(row)
        else:
            right.append(row)
    return left, right

#Best splitpoint for dataset with choice of attributes to avoid redundancy
def get_split_forest(dataset, n_attributes):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    attributes = list()
    while len(attributes) < n_attributes:
        index = randrange(len(dataset[0])-1)
        if index not in attributes:
            attributes.append(index)
    for index in attributes:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            Gini = gini_index(groups, class_values)
            #print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], Gini))
            if Gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], Gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create a function to split a node in function of diffrent parameters
def split_node_forest(node, max_depth, min_size, n_attributes, depth):
    left, right = node['groups']
    #Delete data from node as it is no longer needed
    del(node['groups'])
    #Check whether left node or right node is empty to create a terminal node
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    #Check if we have reached the maximum depth of the tree --> Create terminal node
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    #Process on the both children, by checking if min size is reached first or further split of tree is required
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split_forest(left, n_attributes)
        split_node_forest(node['left'], max_depth, min_size, n_attributes,depth+1)
    #Right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split_forest(right, n_attributes)
        split_node_forest(node['right'], max_depth, min_size, n_attributes,  depth+1)

# Build a decision tree
def build_tree(train, max_depth, n_attributes ,min_size):
    root = get_split_forest(train, n_attributes)
    split_node_forest(root, max_depth, min_size, n_attributes ,1)
    return root

# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)



# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for _ in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)

In [6]:
def main():
    # Test the random forest algorithm on sonar dataset
    seed(2)
    # load and prepare data
    filename = 'sonar-all-data.csv'
    dataset = load_csv(filename)
    # convert string attributes to integers
    for i in range(0, len(dataset[0])-1):
        str_column_to_float(dataset, i)
    # convert class column to integers
    str_column_to_int(dataset, len(dataset[0])-1)
    # evaluate algorithm
    n_folds = 10
    max_depth = 10
    min_size = 1.0
    sample_size = 3.0
    n_features = int(sqrt(len(dataset[0])-1))
    for n_trees in [1, 5, 10]:
        scores = evaluate_algorithm_cv(dataset, random_forest, n_folds, getAccuracy,max_depth, min_size, sample_size, n_trees, n_features)
        print('Trees: %d' % n_trees)
        print('Scores: %s' % scores)
        print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

main()

Trees: 1
Scores: [55.00000000000001, 70.0, 40.0, 60.0, 80.0, 70.0, 75.0, 50.0, 80.0, 80.0]
Mean Accuracy: 66.000%
Trees: 5
Scores: [70.0, 65.0, 90.0, 80.0, 65.0, 65.0, 80.0, 80.0, 75.0, 55.00000000000001]
Mean Accuracy: 72.500%
Trees: 10
Scores: [75.0, 50.0, 80.0, 65.0, 65.0, 80.0, 85.0, 75.0, 85.0, 65.0]
Mean Accuracy: 72.500%
